In [26]:

import bisect
import json
import pandas as pd


from pyspark.sql import DataFrame, Row, SparkSession
from pyspark.sql.types import StructType, StructField, StringType, TimestampNTZType, ArrayType, DataType
from pyspark.sql.functions import from_json, to_json, udf, collect_list
from typing import Dict, Iterator, List, Union

In [27]:
span_log_schema = StructType(
        [
            StructField("trace_id", StringType(), False),
            StructField("span_id", StringType(), False),
            StructField("parent_id", StringType(), False),
            StructField("user_id", StringType(), True),
            StructField("session_id", StringType(), True),
            StructField("span_type", StringType(), False),
            StructField("start_time", TimestampNTZType(), False),
            StructField("end_time", TimestampNTZType(), False),
            StructField("name", StringType(), False),
            StructField("status", StringType(), False),
            StructField("framework", StringType(), False),
            StructField("input", StringType(), False),
            StructField("output", StringType(), False),
            StructField("attributes", StringType(), False),
            StructField("events", ArrayType(DataType()), False),
            StructField("links", ArrayType(DataType()), False),
        ]
    )

In [16]:
def init_spark() -> SparkSession:
    """Get or create spark session."""
    spark = SparkSession.builder.appName("AccessParquetFiles").getOrCreate()
    return spark

spark = init_spark()

In [20]:
df = spark.read.json("./sample_preprocessed_span.jsonl")
df.show()

+--------------------+--------------------+-------------------+--------------------+--------------------+----------------+--------------------+------------+--------------------+-----------+--------------------+--------------------+--------------------+
|               agent|     collectdatatype|       contentrange|       correlationid|                data| datacontenttype|                  id|modelversion|              source|specversion|                time|                type|          xrequestid|
+--------------------+--------------------+-------------------+--------------------+--------------------+----------------+--------------------+------------+--------------------+-----------+--------------------+--------------------+--------------------+
|azureml-ai-monito...|pandas.core.frame...|    bytes 0-867/868|ee3c0e82-7edf-480...|[{{NULL, promptfl...|application/json|6fee5e87-e702-40f...|     default|/subscriptions/96...|        1.0|2024-02-05T15:00:22Z|azureml.inference...|ee3c0e82-7

In [42]:
logs_schema = StructType([
        StructField('attributes', StringType(), True),
        StructField('end_time', StringType(), True),
        StructField('events', ArrayType(StringType(), True), True), 
        StructField('input', StringType(), True), 
        StructField('links', ArrayType(StringType(), True), True), 
        StructField('name', StringType(), True), 
        StructField('output', StringType(), True), 
        StructField('parent_id', StringType(), True), 
        StructField('session_id', StringType(), True), 
        StructField('span_id', StringType(), True), 
        StructField('span_type', StringType(), True), 
        StructField('start_time', StringType(), True), 
        StructField('status', StringType(), True), 
        StructField('trace_id', StringType(), True),
        StructField('user_id', StringType(), True)
])

logs_data = df.select('data').collect()
print(logs_data)

[Row(data=[Row(attributes=Row(flow_id=None, framework='promptflow', function=None, inputs='{\n  "question": "what\'s Attention?",\n  "chat_history": []\n}', line_run_id=None, node_name=None, output='{\n  "output": "Attention is a mechanism used in machine learning models to focus on specific parts of input data when making predictions or generating output. It allows the model to selectively weigh different parts of the input, giving more importance to certain aspects and ignoring others. (Source: 1706.03762.pdf)"\n}', retrieval.documents=None, retrieval.query=None, root_run_id=None, span_type='Flow'), end_time='2024-02-05T15:00:18.791564Z', events=[], input='{\n  "question": "what\'s Attention?",\n  "chat_history": []\n}', links=[], name='promptflow.flow', output='{\n  "output": "Attention is a mechanism used in machine learning models to focus on specific parts of input data when making predictions or generating output. It allows the model to selectively weigh different parts of the i

In [43]:
span_logs = spark.createDataFrame(data=logs_data, schema=list[str])

PySparkValueError: [CANNOT_DETERMINE_TYPE] Some of types cannot be determined after inferring.

In [15]:
traces = span_logs.groupBy(span_logs.trace_id)

AnalysisException: [INVALID_USAGE_OF_STAR_OR_REGEX] Invalid usage of '*' in expression `collect_list`.